# "[Crawling] browserless 크롤링"
> "requests"

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [browserless,request, 크롤링]
- author: 도형준

## requests

- requests는 브라우저 없이 파이썬에서 다이렉트 서버에 http 요청을 넣습니다
- 요청을 받은 페이지는 해당 페이지의 소스코드를 파이썬 내부로 전송해줍니다
- 단, 버튼 클릭이나 광고 닫기 등의 작업은 애초에 브라우저가 없기 때문에 불가능하거
- 오로지 특정 주소로 접속했을 때, 최초에 전송되는 페이지 코드만 가져올 수 있습니다.
- 대신 물리 브라우저를 직접 켜지 않고, 명목상으로 요청만 하기 때문에
- 속도나 리소스면에서 훨씬 우월합니다

- 1.내가 브라우저로 특정 서버 주소를 쳐서 접속시도를 합니다
- 2.서버에 내가 접속한다는 요청이 들어갑니다
- 3.서버가 요청에 응답 하면서 response 데이터를 함께 전송합니다
- 4.브라우저가 받은 데이터를 해석해서 내 PC에 반영합니다
    - 4-1.웹사이트 접속이었으면 소스코드를 받아와서 사람이 볼 수 있도록 자동으로 그려준다(렌더링)
    - 4-2.게임이면 해당 명령어가 들어가서 게임 상황에 반영됩니다

### 필요한 라이브러리 import

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import codecs
import pandas as pd

In [2]:
# 특정 주소에 대한 요청은 requests.get('주소')로 합니다.
# 크롬이 아닌 파이썬을 이용함
# 요청에 대한 서버의 응답(response)는 req 변수에 저장되었다.
req = requests.get("https://www.naver.com")

In [3]:
# html 소스코드 읽어오기(selenium의 .text와 문법은 같으나 기능이 다름에 주의)
# req.text를 이용하면 driver.page_source와 동일한 코드를 받아옵니다
source = req.text

In [4]:
# http 상태코드 가져오기
req.status_code

200

## 알라딘 베스트 셀러 7페이지 크롤링

### 코드

In [5]:
# requests
req = requests.get("https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=Bestseller&BranchType=1&CID=0&page=7&cnt=1000&SortOrder=1")
source = req.text

In [6]:
# Parsing
parsed_source = BeautifulSoup(source, "html.parser")

In [7]:
aladdin_author_list=[]
aladdin_title_list=[]
aladdin_price_list=[]

span_title_list = parsed_source.find_all("a",class_='bo3')

for title in span_title_list:
    i = title.text
    aladdin_title_list.append(i)

# 저자
author_list = [] # 저자가 여러명이라 책 별로 분리하기 위해 새로운 빈 리스트 생성
for num2 in range(len(parsed_source.find_all("div",class_='ss_book_box'))):
    span_author_list = parsed_source.find_all("div",class_='ss_book_box')[num2].find_all("li")[2].find_all("a")[:-1]
    for num3 in range(len(span_author_list)):
        j = span_author_list[num3]
        author = j.text
        author_list.append(author)
    aladdin_author_list.append(author_list)
    author_list = []

# 가격
for num4 in range(len(parsed_source.find_all("div",class_='ss_book_box'))):
    try:
        span_price_list =  parsed_source.find_all("div",class_='ss_book_box')[num4].find_all("li")[3].find("span",class_='ss_p2')
        k = span_price_list.text
        aladdin_price_list.append(k)
    except:
        span_price_list =  parsed_source.find_all("div",class_='ss_book_box')[num4].find_all("li")[2].find("span",class_='ss_p2')
        k = span_price_list.text
        aladdin_price_list.append(k)

In [8]:
print(len(aladdin_title_list))
print(aladdin_title_list,'\n')
print(len(aladdin_author_list))
print(aladdin_author_list,'\n')
print(len(aladdin_price_list))
print(aladdin_price_list)

50
['세상 쉬운 영문법', '어떻게 말해줘야 할까 (60만 부 기념 한정판 리커버)', '브라질에 비가 내리면 스타벅스 주식을 사라', '재수사 2', '뇌, 욕망의 비밀을 풀다', '임신 출산 육아 대백과 (2022~2023년 개정판)', '부동산 경매로 1년 만에 꼬마빌딩주 되다', '사로잡는 얼굴들', '부부 이상, 연인 미만. 1~2 합본판', '그들의 말 혹은 침묵', '데뷔 못 하면 죽는 병 걸림 1부 초판 한정 굿즈박스 세트', '미움받을 용기 (반양장)', '체리새우 : 비밀글입니다', '정의란 무엇인가', '나는 당신이 행복했으면 좋겠습니다', '어디로 가세요 펀자이씨?', '부모의 말', '혼자 공부하는 컴퓨터 구조 + 운영체제', '우리가 빛의 속도로 갈 수 없다면', '당신이 옳다', '2023 선재국어 기출실록 (해설 통합형) 세트 - 전4권', '이게 다 호르몬 때문이야', '지리의 힘', '마지막 몰입', '송사무장의 부동산 경매의 기술', '사랑의 기술', '알고 있다는 착각', '신경 청소 혁명', '영어 감정 표현 사전', '슬로우 미러클 영어 그림책 느리게 100권 읽기의 힘', '토끼전 : 꾀주머니 배 속에 차고 계수나무에 간 달아 놓고', '리틀 라이프 1', '타자들의 생태학', '2023 문동균 한국사 한 권으로 모든 것을 정리하는 빈칸노트', '오늘도 고바야시 서점에 갑니다', '2023 전한길 한국사 3.0 기출문제집 - 전2권', '협력의 유전자', '거꾸로 읽는 세계사', '스파이 패밀리 2', '스파이 패밀리 8', '포켓몬스터 썬&문 포켓몬 전국대도감', '프레임', '법륜 스님의 행복', '나의 히어로 아카데미아 35', '카구야 님은 고백받고 싶어 24', '지극히 사적인 네팔', '2023 심우철 문법 풀이 전략서', '투자에도 순서가 있다', '전쟁과 약, 기나긴 악연의 역사', '사회사상의 역사'] 

50
[['윤여홍'], ['오은영', '차상미'], ['피터 나바로', '이창식', '윤지호']

- 저자에서 빈 곳이 생기는 것으로 보아, except 케이스를 더 추가해야 될 것으로 판단 됨

### txt 파일로 정리하기

In [9]:
f = codecs.open("C:/Users/dhj98/playdata/crawler/알라딘순회_result.txt", mode="w")
f.write("%s,%s,%s,%s\n"%('순위','제목','저자','가격'))
for i in range(len(aladdin_price_list)):
    f.write("%s 순위, %s,%s,%s\n" % ((i+1),
                                                   aladdin_title_list[i].replace(',','/'),
                                                   str(aladdin_author_list[i]).replace(',','/'),
                                                   aladdin_price_list[i].replace(',', '')))
f.close()

### txt to df

In [11]:
df = pd.read_csv("C:/Users/dhj98/playdata/crawler/알라딘_requests_result.txt",encoding='cp949')
df.head()

,순위,제목,저자,가격
0,1 순위,세상 쉬운 영문법,['윤여홍'],17820원
1,2 순위,어떻게 말해줘야 할까 (60만 부 기념 한정판 리커버),['오은영'/ '차상미'],15750원
2,3 순위,브라질에 비가 내리면 스타벅스 주식을 사라,['피터 나바로'/ '이창식'/ '윤지호'],16200원
3,4 순위,재수사 2,['장강명'],14400원
4,5 순위,뇌/ 욕망의 비밀을 풀다,['한스-게오르크 호이젤'/ '강영옥'/ '김신종'/ '한윤진'],16200원
